In [62]:
import pandas as pd
import os
from ftplib import FTP

def connect_to_ftp(path, filename):
    ftp_path_root = 'ftp.ncdc.noaa.gov'
    # Access NOAA FTP server
    ftp = FTP(ftp_path_root)
    message = ftp.login()  # No credentials needed
    print(message)
    ftp.cwd(path) 
    print(ftp.pwd())
#     print(os.listdir(ftp.pwd()))
    handle = open("./" + filename, 'wb')
    ftp.retrbinary('RETR %s' % filename, handle.write)
#     with open('./'+filename, 'w') as fobj:
#         ftp.retrbinary('RETR %s' % filename, fobj.write)        
    ftp.quit()
    return ftp


In [63]:
ftp_path_dly = 'pub/data/ghcn/daily/'
ftp_filename = 'ghcnd_hcn.tar.gz'

In [64]:
ftp = connect_to_ftp(ftp_path_dly, ftp_filename)

230-****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******
 ** This is a United States Department of Commerce computer     **
 ** system, which may be accessed and used only for             **
 ** official Government business by authorized personnel.       **
 ** Unauthorized access or use of this computer system may      **
 ** subject violators to criminal, civil, and/or administrative **
 ** action.  All information on this computer system may be     **
 ** intercepted, recorded, read, copied, and disclosed by and   **
 ** to authorized personnel for official purposes, including    **
 ** criminal investigations.  Access or use of this computer    **
 ** system by any person, whether authorized or unauthorized,   **
 ** constitutes consent to these terms.                         **
 ****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******
230 Anonymous access granted, restrictions apply
/pub/data/ghcn/daily


In [68]:
filePaths = os.listdir('./ghcnd_hcn/ghcnd_hcn')

In [17]:
# for f in filePaths[0:]:
#     data = pd.read_csv('./ghcnd_hcn/'+f, header=None, delimiter = ' ', na_values='-9999', error_bad_lines=False)

b'Skipping line 3: expected 107 fields, saw 109\nSkipping line 4: expected 107 fields, saw 112\nSkipping line 7: expected 107 fields, saw 127\nSkipping line 8: expected 107 fields, saw 153\nSkipping line 9: expected 107 fields, saw 175\nSkipping line 10: expected 107 fields, saw 187\nSkipping line 11: expected 107 fields, saw 187\nSkipping line 13: expected 107 fields, saw 124\nSkipping line 14: expected 107 fields, saw 132\nSkipping line 15: expected 107 fields, saw 176\nSkipping line 16: expected 107 fields, saw 184\nSkipping line 17: expected 107 fields, saw 184\nSkipping line 19: expected 107 fields, saw 125\nSkipping line 20: expected 107 fields, saw 126\nSkipping line 21: expected 107 fields, saw 177\nSkipping line 22: expected 107 fields, saw 187\nSkipping line 23: expected 107 fields, saw 187\nSkipping line 24: expected 107 fields, saw 125\nSkipping line 25: expected 107 fields, saw 133\nSkipping line 26: expected 107 fields, saw 179\nSkipping line 27: expected 107 fields, saw 

b'Skipping line 3: expected 114 fields, saw 152\nSkipping line 4: expected 114 fields, saw 178\nSkipping line 5: expected 114 fields, saw 131\nSkipping line 7: expected 114 fields, saw 137\nSkipping line 8: expected 114 fields, saw 168\nSkipping line 9: expected 114 fields, saw 145\nSkipping line 10: expected 114 fields, saw 127\nSkipping line 11: expected 114 fields, saw 117\nSkipping line 12: expected 114 fields, saw 182\nSkipping line 13: expected 114 fields, saw 133\nSkipping line 14: expected 114 fields, saw 143\nSkipping line 15: expected 114 fields, saw 151\nSkipping line 16: expected 114 fields, saw 182\nSkipping line 17: expected 114 fields, saw 125\nSkipping line 18: expected 114 fields, saw 147\nSkipping line 19: expected 114 fields, saw 116\nSkipping line 20: expected 114 fields, saw 124\nSkipping line 21: expected 114 fields, saw 126\nSkipping line 22: expected 114 fields, saw 150\nSkipping line 23: expected 114 fields, saw 125\nSkipping line 24: expected 114 fields, saw 1

ParserError: Too many columns specified: expected 153 and found 130

In [70]:
#Get raw data years of interest and variables of interest
desiredYears = set(['2015', '2016', '2017', '2018', '2019'])
weatherTypes = set(['PRCP', 'SNWD', 'SNOW', 'TMAX', 'TMIN'])

for f in filePaths:
    with open('./ghcnd_hcn/ghcnd_hcn/'+f, 'r') as fin:
        station = f[:-4]
        toWrite = []
        line = fin.readline()
        while line:
            line = line.split()
            station = str(line[0][:11])
            year = str(line[0][11:15])
            month = str(line[0][15:17])
            weatherType = str(line[0][17:])
            restOfLine = line[1:]
            if len(weatherType) > 4:
                weatherType = weatherType[:4]
                other = weatherType[4:]
                restOfLine = [other] + restOfLine
            if year in desiredYears and weatherType in weatherTypes:
                toWrite.append('\t'.join([station]+[year]+[month]+[weatherType]+restOfLine)+'\n')
            line = fin.readline()
        with open('./weatherOfInterest','a') as fout:
            for l in toWrite:
                fout.write(l)
            toWrite=[]

In [159]:
# print([(x[0],x[-1]) for x in [longlist[i:i + 10000] for i in range(0, len(longlist), 10000)]])
# ["{0:0=3d}".format(x) for x in range(30)]

[(0, 9999), (10000, 19999), (20000, 29999), (30000, 39999), (40000, 49999), (50000, 59999), (60000, 69999), (70000, 79999), (80000, 89999), (90000, 99999), (100000, 109999), (110000, 119999), (120000, 129999), (130000, 139999), (140000, 149999), (150000, 159999), (160000, 169999), (170000, 179999), (180000, 189999), (190000, 199999), (200000, 209999), (210000, 219999), (220000, 229999), (230000, 234031)]


In [71]:
#Clean data and save
with open('./weatherOfInterest','r') as fin:
    lines = fin.readlines()
    newLines = [x.replace('-9999','-9999-9999') for x in lines]
    newLines = [x.replace('-9999', '\t-9999') for x in newLines]
    newLines = [x.replace('0T','0 ') for x in newLines]
with open('./weatherOfInterestCleaned','w') as fout:
    for l in newLines:
        l = l.split()
        start = l[0:4]
        end = l[4:]
        end = end[0::2]
        newline = start + end
        while len(newline) < 35:
            newline.append('-9999')
        fout.write('\t'.join(newline)+'\n')

In [72]:
data = pd.read_csv('./weatherOfInterestCleaned', header=None, delimiter = '\t', na_values='-9999', error_bad_lines=False)

In [75]:
# !pip install uszipcode
from calendar import monthrange
from uszipcode import Zipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)
# result = search.by_coordinates(39.122229, -77.133578, radius=30, returns=1)
# result[0].means_of_transportation_to_work_for_workers_16_and_over

  Using cached https://files.pythonhosted.org/packages/34/e1/f828cd05732433d0074a17a2623a8c1281c5dc7fb265fef1d3867692037b/uszipcode-0.2.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ff/f2/a1e6044fe90784e7bbc05286f2e8616aa2ff167f7275f5a6f2df479092c0/pathlib_mate-0.0.15-py2.py3-none-any.whl


In [82]:
station2Zip = {}
stationsWithData = set([x for x in data[0]])
with open('./ghcnd-stations.txt','r') as fin:
    lines = fin.readlines()
    lines = [l.split()[0:3] for l in lines]
    lines = [x for x in lines if x[0][0:2]=='US']
    for l in lines:
        stationName = l[0]
        if stationName in stationsWithData:
            lat = float(l[1])
            lng = float(l[2])     
            result = search.by_coordinates(lat,lng, radius=30, returns=1)
            if len(result) < 1:
                result = search.by_coordinates(lat,lng, radius=100, returns=1)
            if len(result) == 1:
                station2Zip[stationName] = result[0].zipcode
            else:
                print("Station %s not found" %stationName)

In [83]:
data['ZIP'] = data[0].apply(lambda x: station2Zip[x])
data['ZIP3'] = data['ZIP'].apply(lambda x: x[:3])

In [84]:
print('We have %d distinct zip codes, and %d distinct ZIP3' %(len(set(data['ZIP'])), len(set(data['ZIP3']))))

We have 974 distinct zip codes, and 504 distinct ZIP3


In [85]:
grouped = data.groupby([0,1,2,3,'ZIP','ZIP3'],as_index=False)

In [86]:
summaryWeather = grouped.mean().groupby([1,2,3,'ZIP3'],as_index=False).mean()
summaryWeatherImputed = summaryWeather.fillna(method='bfill', axis=1)
summaryWeatherImputed = summaryWeatherImputed.fillna(method='ffill', axis=1)

In [87]:
summaryWeather.columns = ['Year', 'Month', 'Type', 'ZIP3']+[x for x in range(1,32)]
summaryWeatherImputed.columns = ['Year', 'Month', 'Type', 'ZIP3']+[x for x in range(1,32)]

In [88]:
maxMin = summaryWeatherImputed[(summaryWeather['Type']=='TMAX') | (summaryWeather['Type']=='TMIN')]

In [89]:
maxMinCount = maxMin.drop(columns=['Type']).groupby(['Year','Month','ZIP3'],as_index=False).count()

In [90]:
maxMinSum = maxMin.drop(columns=['Type']).groupby(['Year','Month','ZIP3'],as_index=False).sum()

In [91]:
# maxMin2[maxMin2[1]<2]
ask = maxMinSum.iloc[:,3:] / maxMinCount.iloc[:,3:]
# maxMin2[maxMin2['ZIP3']=='100']
ask.shape
maxMinSum.shape

(24961, 34)

In [92]:
maxMin = pd.concat([maxMinSum.iloc[:,0:3],ask], axis=1)
maxMin['Type'] = 'TAVG'
maxMin[maxMin['ZIP3']=='100']

,Year,Month,ZIP3,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,Type
30,2015,1,100,6.0,36.5,31.0,91.5,17.0,-63.0,-88.0,...,9.0,22.5,25.5,-30.0,-38.0,-38.5,-24.5,-19.0,-68.5,TAVG
510,2015,2,100,-22.0,-44.0,-68.5,9.0,-21.5,-68.5,3.0,...,-49.5,-99.0,-19.0,-30.0,-43.5,-60.5,-60.5,-60.5,-60.5,TAVG
983,2015,3,100,-24.0,6.0,-13.5,44.5,-13.5,-68.5,-22.0,...,-8.0,20.0,52.5,111.5,58.5,6.0,20.0,72.0,50.0,TAVG
1463,2015,4,100,53.0,122.0,164.0,106.0,108.5,114.0,111.5,...,80.5,75.0,100.0,128.0,130.5,158.5,178.0,141.5,141.5,TAVG
1944,2015,5,100,133.0,161.0,186.5,216.5,241.5,189.0,200.0,...,152.5,202.5,236.0,252.5,252.5,252.5,238.5,244.0,222.5,TAVG
2428,2015,6,100,125.0,114.0,161.0,152.5,169.5,200.0,180.5,...,280.5,244.5,233.0,239.0,180.5,197.5,208.0,239.0,239.0,TAVG
2909,2015,7,100,242.0,236.5,233.5,222.5,241.5,252.5,275.0,...,250.0,258.5,258.5,277.5,258.5,294.5,306.0,275.0,269.5,TAVG
3390,2015,8,100,278.0,267.0,280.5,261.5,264.0,238.5,247.5,...,252.5,267.0,275.0,244.0,236.0,225.0,250.0,272.0,286.0,TAVG
3873,2015,9,100,283.0,280.5,286.0,258.0,238.5,244.5,272.0,...,208.5,217.0,208.0,191.5,175.0,219.5,252.5,205.5,205.5,TAVG
4356,2015,10,100,155.5,114.0,111.0,144.5,152.5,169.5,186.0,...,127.5,91.5,144.5,111.0,111.0,161.5,183.5,125.0,94.5,TAVG


In [93]:
summaryWeatherImputed[(summaryWeatherImputed['ZIP3']=='440') & ((summaryWeatherImputed['Type']=='TMAX') | (summaryWeatherImputed['Type']=='TMIN'))]

,Year,Month,Type,ZIP3,1,2,3,4,5,6,...,22,23,24,25,26,27,28,29,30,31
106269,2018,12,TMAX,440,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108450,2019,1,TMAX,440,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
108908,2019,1,TMIN,440,-167.0,-167.0,-167.0,-167.0,-167.0,-167.0,...,-167.0,-167.0,-167.0,-167.0,-167.0,-167.0,-167.0,-167.0,-167.0,-167.0
110635,2019,2,TMAX,440,50.0,50.0,50.0,50.0,50.0,50.0,...,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0
115195,2019,4,TMIN,440,33.0,33.0,33.0,33.0,33.0,33.0,...,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0
116613,2019,5,TMAX,440,72.0,72.0,72.0,72.0,72.0,72.0,...,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0
117020,2019,5,TMIN,440,67.0,67.0,67.0,67.0,67.0,67.0,...,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0


In [94]:
maxMin[maxMin['ZIP3']=='100']

,Year,Month,ZIP3,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,Type
30,2015,1,100,6.0,36.5,31.0,91.5,17.0,-63.0,-88.0,...,9.0,22.5,25.5,-30.0,-38.0,-38.5,-24.5,-19.0,-68.5,TAVG
510,2015,2,100,-22.0,-44.0,-68.5,9.0,-21.5,-68.5,3.0,...,-49.5,-99.0,-19.0,-30.0,-43.5,-60.5,-60.5,-60.5,-60.5,TAVG
983,2015,3,100,-24.0,6.0,-13.5,44.5,-13.5,-68.5,-22.0,...,-8.0,20.0,52.5,111.5,58.5,6.0,20.0,72.0,50.0,TAVG
1463,2015,4,100,53.0,122.0,164.0,106.0,108.5,114.0,111.5,...,80.5,75.0,100.0,128.0,130.5,158.5,178.0,141.5,141.5,TAVG
1944,2015,5,100,133.0,161.0,186.5,216.5,241.5,189.0,200.0,...,152.5,202.5,236.0,252.5,252.5,252.5,238.5,244.0,222.5,TAVG
2428,2015,6,100,125.0,114.0,161.0,152.5,169.5,200.0,180.5,...,280.5,244.5,233.0,239.0,180.5,197.5,208.0,239.0,239.0,TAVG
2909,2015,7,100,242.0,236.5,233.5,222.5,241.5,252.5,275.0,...,250.0,258.5,258.5,277.5,258.5,294.5,306.0,275.0,269.5,TAVG
3390,2015,8,100,278.0,267.0,280.5,261.5,264.0,238.5,247.5,...,252.5,267.0,275.0,244.0,236.0,225.0,250.0,272.0,286.0,TAVG
3873,2015,9,100,283.0,280.5,286.0,258.0,238.5,244.5,272.0,...,208.5,217.0,208.0,191.5,175.0,219.5,252.5,205.5,205.5,TAVG
4356,2015,10,100,155.5,114.0,111.0,144.5,152.5,169.5,186.0,...,127.5,91.5,144.5,111.0,111.0,161.5,183.5,125.0,94.5,TAVG


In [95]:
cleanWeather = pd.concat([summaryWeatherImputed[(summaryWeather['Type']=='PRCP') | (summaryWeather['Type']=='SNOW')], maxMin])

/Users/juanmazam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [96]:
cleanWeather = cleanWeather[['Year', 'Month', 'Type', 'ZIP3']+[x for x in range(1,32)]]

In [97]:
cleanWeather.to_csv('cleanWeather')

In [98]:
cleanWeather.shape

(71386, 35)

In [99]:
cleanWeather[cleanWeather['Type']=='TAVG']

,Year,Month,Type,ZIP3,1,2,3,4,5,6,...,22,23,24,25,26,27,28,29,30,31
0,2015,1,TAVG,010,-64.000000,-36.000000,-16.500000,-33.500000,-5.500000,-30.500000,...,-66.500000,-47.500000,-39.000000,-16.500000,-50.000000,-86.000000,-81.000000,-111.000000,-105.500000,-64.000000
1,2015,1,TAVG,018,-55.250000,-6.750000,-23.500000,23.500000,14.000000,-82.000000,...,-23.750000,-19.500000,-16.750000,-18.250000,-79.250000,-98.500000,-97.250000,-88.750000,-44.250000,-80.500000
2,2015,1,TAVG,020,-55.500000,-16.750000,-25.000000,1.250000,39.250000,-68.000000,...,-24.750000,-28.000000,-18.000000,-7.000000,-65.500000,-72.250000,-98.500000,-94.750000,-31.750000,-66.500000
3,2015,1,TAVG,023,-36.000000,-11.000000,-3.000000,33.500000,39.000000,-69.500000,...,-11.000000,-14.000000,-16.500000,-6.000000,-44.500000,-61.000000,-61.000000,-55.500000,-11.000000,-58.000000
4,2015,1,TAVG,028,-37.250000,11.500000,-13.750000,58.500000,32.250000,-73.250000,...,3.000000,-16.500000,4.250000,1.500000,-46.500000,-66.250000,-71.500000,-81.500000,-21.750000,-67.500000
5,2015,1,TAVG,034,-86.000000,-58.500000,-44.500000,-53.000000,19.000000,-69.500000,...,-75.000000,-47.500000,-61.000000,-19.500000,-66.500000,-103.000000,-105.500000,-133.000000,-113.500000,-72.000000
6,2015,1,TAVG,035,-144.500000,-91.500000,-150.000000,-83.000000,-16.500000,-166.500000,...,-155.500000,-97.500000,-97.000000,-69.500000,-194.000000,-200.000000,-147.000000,-216.500000,-153.000000,-125.000000
7,2015,1,TAVG,037,-58.000000,-22.000000,-80.500000,-30.500000,-22.500000,-125.000000,...,-55.500000,-66.500000,-50.000000,-30.500000,-130.500000,-100.000000,-97.000000,-116.500000,-67.000000,-113.500000
8,2015,1,TAVG,038,-72.000000,-25.000000,-58.500000,-72.000000,-16.500000,-86.000000,...,-22.500000,-38.500000,-44.500000,-19.000000,-119.000000,-116.500000,-116.500000,-133.000000,-42.000000,-75.000000
9,2015,1,TAVG,041,-38.500000,-11.000000,-44.000000,-16.000000,-38.500000,-121.000000,...,-13.500000,-32.500000,-19.000000,-44.000000,-113.000000,-99.000000,-85.000000,-101.500000,-30.000000,-96.500000


In [102]:
cleanWeather[(cleanWeather['ZIP3'] == '100')&(cleanWeather['Type']=='TAVG')]

,Year,Month,Type,ZIP3,1,2,3,4,5,6,...,22,23,24,25,26,27,28,29,30,31
30,2015,1,TAVG,100,6.0,36.5,31.0,91.5,17.0,-63.0,...,19.5,9.0,22.5,25.5,-30.0,-38.0,-38.5,-24.5,-19.0,-68.5
510,2015,2,TAVG,100,-22.0,-44.0,-68.5,9.0,-21.5,-68.5,...,30.5,-49.5,-99.0,-19.0,-30.0,-43.5,-60.5,-60.5,-60.5,-60.5
983,2015,3,TAVG,100,-24.0,6.0,-13.5,44.5,-13.5,-68.5,...,20.0,-8.0,20.0,52.5,111.5,58.5,6.0,20.0,72.0,50.0
1463,2015,4,TAVG,100,53.0,122.0,164.0,106.0,108.5,114.0,...,147.5,80.5,75.0,100.0,128.0,130.5,158.5,178.0,141.5,141.5
1944,2015,5,TAVG,100,133.0,161.0,186.5,216.5,241.5,189.0,...,180.5,152.5,202.5,236.0,252.5,252.5,252.5,238.5,244.0,222.5
2428,2015,6,TAVG,100,125.0,114.0,161.0,152.5,169.5,200.0,...,258.5,280.5,244.5,233.0,239.0,180.5,197.5,208.0,239.0,239.0
2909,2015,7,TAVG,100,242.0,236.5,233.5,222.5,241.5,252.5,...,250.0,250.0,258.5,258.5,277.5,258.5,294.5,306.0,275.0,269.5
3390,2015,8,TAVG,100,278.0,267.0,280.5,261.5,264.0,238.5,...,253.0,252.5,267.0,275.0,244.0,236.0,225.0,250.0,272.0,286.0
3873,2015,9,TAVG,100,283.0,280.5,286.0,258.0,238.5,244.5,...,192.0,208.5,217.0,208.0,191.5,175.0,219.5,252.5,205.5,205.5
4356,2015,10,TAVG,100,155.5,114.0,111.0,144.5,152.5,169.5,...,191.5,127.5,91.5,144.5,111.0,111.0,161.5,183.5,125.0,94.5
